# Security-Constrained LOPF with SciGRID

This Jupyter Notebook is also available to download at: <http://www.pypsa.org/examples/scigrid-sclopf.ipynb>  and can be viewed as an HTML page at: <http://pypsa.org/examples/scigrid-sclopf.html>.

In this example, the dispatch of generators is optimised using the security-constrained linear OPF, to guaranteed that no branches are overloaded by certain branch outages.

The data files for this example are in the examples folder of the github repository: <https://github.com/PyPSA/PyPSA>.

## Data sources and health warnings

See the separate notebook at <http://www.pypsa.org/examples/add_load_gen_trafos_to_scigrid.ipynb>.


In [1]:

from __future__ import print_function, division, absolute_import

import pypsa, os


In [4]:
csv_folder_name = "scigrid-de/scigrid-with-load-gen-trafos/"

network = pypsa.Network(csv_folder_name=csv_folder_name)


Importing PyPSA from older version of PyPSA than current version 0.16.0.
Please read the release notes at https://pypsa.org/doc/release_notes.html
carefully to prepare your network for import.

INFO:pypsa.io:Imported network  has buses, generators, lines, loads, storage_units, transformers


In [5]:
#There are some infeasibilities without line extensions                                                                                  
for line_name in ["316","527","602"]:
    network.lines.loc[line_name,"s_nom"] = 1200


now = network.snapshots[0]

In [6]:
branch_outages = network.lines.index[:15]

print("Performing security-constrained linear OPF:")

network.sclopf(now,branch_outages=branch_outages)
print("Objective:",network.objective)

Performing security-constrained linear OPF:


INFO:numexpr.utils:NumExpr defaulting to 8 threads.
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:310: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/pypsa/contingency.py:80: RuntimeWarning: divide by zero encountered in true_divide
  denominator = csr_matrix((1/(1-np.diag(branch_PTDF)),(r_[:num_branches],r_[:num_branches])))
INFO:pypsa.opf:Building pyomo model using `angles` formulation
INFO:pypsa.opf:Solving model using glpk
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 347887.092552073
  Upper bound: 347887.092552073
  Number of objectives: 1
  Number of constraints: 31947
  Number of variables: 3071
  Number of nonzeros: 59821
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 4.3303422927856445
# ----------------------------------------------------------
#   Solution Information
# -----

In [7]:


#For the PF, set the P to the optimised P
network.generators_t.p_set = network.generators_t.p_set.reindex(columns=network.generators.index)
network.generators_t.p_set.loc[now] = network.generators_t.p.loc[now]
network.storage_units_t.p_set = network.storage_units_t.p_set.reindex(columns=network.storage_units.index)
network.storage_units_t.p_set.loc[now] = network.storage_units_t.p.loc[now]

#Check no lines are overloaded with the linear contingency analysis

p0_test = network.lpf_contingency(now,branch_outages=branch_outages)

INFO:pypsa.pf:Performing linear load-flow on AC sub-network SubNetwork 0 for snapshot(s) DatetimeIndex(['2011-01-01'], dtype='datetime64[ns]', freq=None)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:310: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/pypsa/contingency.py:80: RuntimeWarning: divide by zero encountered in true_divide
  denominator = csr_matrix((1/(1-np.diag(branch_PTDF)),(r_[:num_branches],r_[:num_branches])))


In [8]:
p0_test

base    (Line, 1)    (Line, 2)    (Line, 3)  \
            name                                                       
Line        1      -68.803462     0.000000   -68.803462   -68.803462   
            2      190.707297   190.707297     0.000000   212.790071   
            3      325.824500   325.824500   334.145556     0.000000   
            4     -750.815240  -750.815240  -724.824608  -773.225254   
            5     1069.888707  1069.888707  1045.960042  1054.623201   
...                       ...          ...          ...          ...   
Transformer 404      3.995926     3.995926     3.998271     3.994990   
            413     94.603092    94.603092    94.650025    94.624896   
            421     52.977690    52.977690    53.170624    53.069459   
            450     82.518670    82.518670    82.460261    82.496273   
            458     83.475451    83.475451    83.415986    83.452656   

                   (Line, 4)   (Line, 5)    (Line, 6)    (Line, 7)  \
            name                                                     
Line        1     -68.803462  -68.803462   -68.803462   -68.803462   
            2      13.505587 -122.706583   190.839658   191.016630   
            3     383.397387  250.482736   325.812944   325.797492   
            4       0.000000 -487.570141  -750.782643  -750.739059   
            5     932.859992    0.000000  1067.815642  1065.043871   
...                      ...         ...          ...          ...   
Transformer 404     3.992286    3.984331     3.995920     3.995911   
            413    94.833998   94.575138    94.606061    94.610030   
            421    53.942929   52.873378    52.990054    53.006586   
            450    82.250384   82.130705    82.517163    82.515147   
            458    83.202370   83.080708    83.473917    83.471864   

                    (Line, 8)    (Line, 9)   (Line, 10)   (Line, 11)  \
            name                                                       
Line        1      -68.803462   -68.803462   -68.803462   -68.803462   
            2      190.975588   190.815168   191.158964   154.292840   
            3      325.801075   325.853072   325.946010   313.484611   
            4     -750.749167  -750.921335  -751.264946  -708.345998   
            5     1065.686683  1069.894062  1069.913764  1156.214767   
...                       ...          ...          ...          ...   
Transformer 404      3.995913     3.988666     3.966848     3.993604   
            413     94.609110    94.624523    94.688311    94.841071   
            421     53.002752    53.084125    53.401446    53.971018   
            450     82.515615    82.523638    82.539026    82.337607   
            458     83.472340    83.480509    83.496173    83.291123   

                   (Line, 12)   (Line, 13)   (Line, 14)   (Line, 15)  
            name                                                      
Line        1      -68.803462   -68.803462   -68.803462   -68.803462  
            2      190.026641   190.064387   191.158210   190.758718  
            3      327.596357   327.498098   325.945807   325.839954  
            4     -756.662627  -756.338355  -751.264195  -750.871171  
            5     1134.263578  1130.693620  1069.913723  1069.893834  
...                       ...          ...          ...          ...  
Transformer 404      3.996373     3.996348     3.966897     3.993757  
            413     94.467175    94.474713    94.688169    94.608894  
            421     52.411840    52.443219    53.400739    53.007013  
            450     82.595020    82.590786    82.538992    82.520601  
            458     83.553192    83.548881    83.496138    83.477418  

[948 rows x 16 columns]

In [9]:
#check loading as per unit of s_nom in each contingency

max_loading = abs(p0_test.divide(network.passive_branches().s_nom,axis=0)).describe().loc["max"]

print(max_loading)


base          1.0
(Line, 1)     1.0
(Line, 2)     1.0
(Line, 3)     1.0
(Line, 4)     1.0
(Line, 5)     1.0
(Line, 6)     1.0
(Line, 7)     1.0
(Line, 8)     1.0
(Line, 9)     1.0
(Line, 10)    1.0
(Line, 11)    1.0
(Line, 12)    1.0
(Line, 13)    1.0
(Line, 14)    1.0
(Line, 15)    1.0
Name: max, dtype: float64


In [10]:
import numpy as np
np.testing.assert_array_almost_equal(max_loading,np.ones((len(max_loading))))